In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [3]:
import numpy as np
from tqdm import tqdm
from pathlib import Path
import openslide
import pandas as pd
import pickle

In [4]:
from fastai.callbacks.csv_logger import CSVLogger

In [5]:
from object_detection_fastai.helper.object_detection_helper import *
from object_detection_fastai.helper.wsi_loader import *
from object_detection_fastai.loss.RetinaNetFocalLoss import RetinaNetFocalLoss
from object_detection_fastai.models.RetinaNet import RetinaNet
from object_detection_fastai.callbacks.callbacks import BBLossMetrics, BBMetrics, PascalVOCMetric, PascalVOCMetricByDistance

In [6]:
slides_train = list(set(["03_EIPH_566933 R Turnbull blue.svs", 
                         "22_EIPH_575216 Turnbull blue.svs", 
                         "27_EIPH_571557 berliner blau.svs", 
                         "15_EIPH_568320 Turnbull blue.svs", 
                         "17_EIPH_575796 Berliner Blau.svs", 
                        ]))

slides_val = list(set(["07_EIPH_574882 left lung Turnbull blue.svs", "31_EIPH_568354 Berliner Blau.svs"]))

In [7]:
grade_list = [0, 1]
annotations_path = Path("../../Statistics/EIPH_Annotations.pkl")
annotations = pd.read_pickle(annotations_path)
annotations["image_name"] = [name.replace("tiff","svs") for name in annotations["image_name"]]
annotations = annotations[annotations["grade"].isin(grade_list)]
annotations_train = annotations[annotations["image_name"].isin(slides_train)]
annotations_val = annotations[annotations["image_name"].isin(slides_val)]
annotations_train.head()

,id,image_id,image_set,species,image_name,image_type,grade,vector,unique_identifier,user_id,deleted,last_editor,data_set_name,version
32819,1466753,3554,246,equine,17_EIPH_575796 Berliner Blau.svs,BerlinerBlau,0,"{'x1': 21333, 'x2': 21436, 'y1': 10670, 'y2': ...",17233cc6-7a22-48ed-9332-7eb591ce7259,1,True,1,SDATA,Cluster
32820,1466754,3554,246,equine,17_EIPH_575796 Berliner Blau.svs,BerlinerBlau,0,"{'x1': 20283, 'x2': 20361, 'y1': 7117, 'y2': 7...",7a971ffe-a3a3-4fcf-90e3-f9ec8b09bb4a,1,True,1,SDATA,Cluster
32821,1466755,3554,246,equine,17_EIPH_575796 Berliner Blau.svs,BerlinerBlau,0,"{'x1': 8287, 'x2': 8412, 'y1': 15362, 'y2': 15...",4ec0093b-c83c-4169-a226-f136f6b29de6,1,True,1,SDATA,Cluster
32822,1466756,3554,246,equine,17_EIPH_575796 Berliner Blau.svs,BerlinerBlau,0,"{'x1': 8829, 'x2': 8950, 'y1': 24881, 'y2': 25...",a04af077-74ab-45b3-b51c-85b585e49235,1,True,1,SDATA,Cluster
32823,1466757,3554,246,equine,17_EIPH_575796 Berliner Blau.svs,BerlinerBlau,0,"{'x1': 25522, 'x2': 25596, 'y1': 21891, 'y2': ...",deaacf08-f001-4108-8bb0-c215af5bc87e,1,True,1,SDATA,Cluster


In [8]:
slides_path = Path("../../../Slides")
files = {slide.name: slide for slide in slides_path.rglob("*.svs")  if slide.name in slides_train + slides_val}
files

{'03_EIPH_566933 R Turnbull blue.svs': PosixPath('../../../Slides/Equine/03_EIPH_566933 R Turnbull blue.svs'),
 '07_EIPH_574882 left lung Turnbull blue.svs': PosixPath('../../../Slides/Equine/07_EIPH_574882 left lung Turnbull blue.svs'),
 '15_EIPH_568320 Turnbull blue.svs': PosixPath('../../../Slides/Equine/15_EIPH_568320 Turnbull blue.svs'),
 '17_EIPH_575796 Berliner Blau.svs': PosixPath('../../../Slides/Equine/17_EIPH_575796 Berliner Blau.svs'),
 '22_EIPH_575216 Turnbull blue.svs': PosixPath('../../../Slides/Equine/22_EIPH_575216 Turnbull blue.svs'),
 '27_EIPH_571557 berliner blau.svs': PosixPath('../../../Slides/Equine/27_EIPH_571557 berliner blau.svs'),
 '31_EIPH_568354 Berliner Blau.svs': PosixPath('../../../Slides/Equine/31_EIPH_568354 Berliner Blau.svs')}

In [9]:
tfms = get_transforms(do_flip=True,
                      flip_vert=True,
                      #max_rotate=90,
                      max_lighting=0.0,
                      max_zoom=1.,
                      max_warp=0.0,
                      p_affine=0.5,
                      p_lighting=0.0,
                      #xtra_tfms=xtra_tfms,
                     )
tfms

([RandTransform(tfm=TfmCrop (crop_pad), kwargs={'row_pct': (0, 1), 'col_pct': (0, 1), 'padding_mode': 'reflection'}, p=1.0, resolved={}, do_run=True, is_random=True, use_on_y=True),
  RandTransform(tfm=TfmAffine (dihedral_affine), kwargs={}, p=1.0, resolved={}, do_run=True, is_random=True, use_on_y=True),
  RandTransform(tfm=TfmAffine (rotate), kwargs={'degrees': (-10.0, 10.0)}, p=0.5, resolved={}, do_run=True, is_random=True, use_on_y=True)],
 [RandTransform(tfm=TfmCrop (crop_pad), kwargs={}, p=1.0, resolved={}, do_run=True, is_random=True, use_on_y=True)])

In [10]:
size = 1024 
level = 0
bs = 16
train_images = 2500
val_images = 1500

In [11]:
def get_y_func(x):
    return x.y

In [12]:
anchors = create_anchors(sizes=[(32,32)], ratios=[1], scales=[0.6, 0.7, 0.9, 1.25, 1.5])

In [13]:
for i in range(len(slides_train)):
    
    torch.cuda.empty_cache()

    train_files = []
    val_files = []
    
    experiment_name = "EquineVsEquine-Ablation_{}".format(i)

    for image_name in slides_train[:i+1]:

        annotations = annotations_train[annotations_train["image_name"] == image_name]
        annotations = annotations[annotations["deleted"] == False]

        slide_path = files[image_name]
        labels =  list(annotations["grade"])
        bboxes = [[vector["x1"], vector["y1"], vector["x2"], vector["y2"]] for vector in annotations["vector"]]
        
        train_files.append(SlideContainer(slide_path, y=[bboxes, labels],  level=level, width=size, height=size))

    for image_name in annotations_val["image_name"].unique():

        annotations = annotations_val[annotations_val["image_name"] == image_name]
        annotations = annotations[annotations["deleted"] == False]

        slide_path = files[image_name]
        labels =  list(annotations["grade"])
        bboxes = [[vector["x1"], vector["y1"], vector["x2"], vector["y2"]] for vector in annotations["vector"]]
        
        val_files.append(SlideContainer(slide_path, y=[bboxes, labels],  level=level, width=size, height=size))
    
    train_files = list(np.random.choice(train_files, train_images))
    valid_files = list(np.random.choice(val_files, val_images))
    
    train =  ObjectItemListSlide(train_files, path=slides_path)
    valid = ObjectItemListSlide(valid_files, path=slides_path)
    item_list = ItemLists(slides_path, train, valid)
    lls = item_list.label_from_func(get_y_func, label_cls=SlideObjectCategoryList) #
    lls = lls.transform(tfms, tfm_y=True, size=size)
    data = lls.databunch(bs=bs, collate_fn=bb_pad_collate).normalize()
    
    crit = RetinaNetFocalLoss(anchors)
    encoder = create_body(models.resnet18, True, -2)
    model = RetinaNet(encoder, n_classes=data.train_ds.c, n_anchors=5, sizes=[32], chs=128, final_bias=-4., n_conv=3)
    
    voc = PascalVOCMetricByDistance(anchors, size, [str(i) for i in data.train_ds.y.classes[1:]], radius=40)
    learn = Learner(data, model, loss_func=crit, callback_fns=[BBMetrics, partial(CSVLogger, append=False, filename=experiment_name)], #BBMetrics, ShowGraph
                    metrics=[voc])

    learn.split([model.encoder[6], model.c5top5])
    learn.freeze_to(-2)
    
    learn.unfreeze()
    learn.fit_one_cycle(10, 1e-3)
        
    learn.destroy() 

epoch,train_loss,valid_loss,pascal_voc_metric_by_distance,BBloss,focal_loss,AP-0,AP-1,time
0,0.394625,0.469981,0.571181,0.131774,0.338208,0.667123,0.475239,03:59
1,0.244947,0.426316,0.642593,0.127393,0.298922,0.734633,0.550554,04:02
2,0.229875,0.523494,0.598705,0.142043,0.381451,0.709745,0.487665,03:56
3,0.187930,0.426316,0.647441,0.123364,0.302952,0.769724,0.525158,04:01
4,0.166441,0.332675,0.627622,0.069577,0.263098,0.627055,0.628188,03:54
5,0.144276,0.392556,0.570987,0.080451,0.312106,0.693720,0.448255,03:58
6,0.133030,0.419825,0.535288,0.075463,0.344362,0.680532,0.390044,03:58
7,0.111539,0.516196,0.544989,0.075122,0.441074,0.689359,0.400620,03:56
8,0.100605,0.498322,0.560607,0.072725,0.425597,0.695718,0.425495,03:55
9,0.091086,0.463402,0.587027,0.070138,0.393264,0.706099,0.467956,03:57


this Learner object self-destroyed - it still exists, but no longer usable


epoch,train_loss,valid_loss,pascal_voc_metric_by_distance,BBloss,focal_loss,AP-0,AP-1,time
0,0.389986,0.378609,0.585088,0.087223,0.291386,0.716853,0.453322,04:00
1,0.254566,0.382857,0.507020,0.099350,0.283507,0.711381,0.302658,04:00
2,0.216490,0.358296,0.563435,0.056836,0.301460,0.775993,0.350877,04:02
3,0.199546,0.309446,0.626437,0.070093,0.239353,0.750319,0.502555,03:57
4,0.168729,0.284520,0.778789,0.060919,0.223601,0.805957,0.751622,04:01
5,0.156552,0.366362,0.604016,0.059486,0.306876,0.761148,0.446884,03:58
6,0.132729,0.267646,0.680379,0.053469,0.214177,0.790822,0.569936,03:57
7,0.120724,0.240945,0.663290,0.046640,0.194305,0.795718,0.530862,04:00
8,0.108409,0.222122,0.706652,0.042722,0.179400,0.811886,0.601417,03:57
9,0.100701,0.243562,0.683608,0.043291,0.200271,0.814986,0.552229,04:01


this Learner object self-destroyed - it still exists, but no longer usable


epoch,train_loss,valid_loss,pascal_voc_metric_by_distance,BBloss,focal_loss,AP-0,AP-1,time
0,0.429827,0.398323,0.469358,0.077595,0.320728,0.690828,0.247888,04:00
1,0.271572,0.366619,0.522723,0.073950,0.292669,0.666811,0.378635,03:58
2,0.271465,0.469208,0.463445,0.092414,0.376794,0.647652,0.279237,03:57
3,0.227358,0.378657,0.589057,0.061022,0.317635,0.694893,0.483221,03:54
4,0.207192,0.264571,0.651975,0.062324,0.202247,0.788223,0.515726,03:57
5,0.182089,0.348143,0.621783,0.057407,0.290736,0.807716,0.435851,04:03
6,0.163219,0.338124,0.573143,0.059955,0.278169,0.748257,0.398029,03:58
7,0.147729,0.274743,0.622777,0.050513,0.224230,0.793509,0.452044,03:59
8,0.133959,0.238751,0.656593,0.044658,0.194093,0.819400,0.493786,03:58
9,0.132056,0.251887,0.646760,0.044619,0.207268,0.814363,0.479158,03:59


this Learner object self-destroyed - it still exists, but no longer usable


epoch,train_loss,valid_loss,pascal_voc_metric_by_distance,BBloss,focal_loss,AP-0,AP-1,time
0,0.462862,0.381607,0.468645,0.086614,0.294993,0.680616,0.256673,04:01
1,0.317164,0.782911,0.654308,0.075586,0.707326,0.609322,0.699294,04:02
2,0.278327,0.282271,0.729619,0.075719,0.206551,0.800455,0.658782,03:58
3,0.263039,0.377410,0.646568,0.086442,0.290968,0.717459,0.575678,03:55
4,0.240334,0.254560,0.729209,0.063638,0.190922,0.770224,0.688194,03:59
5,0.212071,0.384046,0.451229,0.059437,0.324609,0.637437,0.265020,04:00
6,0.186142,0.246316,0.684626,0.053710,0.192606,0.800479,0.568774,03:58
7,0.165313,0.245908,0.691610,0.043646,0.202262,0.802826,0.580394,03:56
8,0.152947,0.249205,0.659828,0.044278,0.204927,0.810640,0.509017,03:56
9,0.144854,0.256848,0.637716,0.044533,0.212314,0.780155,0.495277,03:57


this Learner object self-destroyed - it still exists, but no longer usable


epoch,train_loss,valid_loss,pascal_voc_metric_by_distance,BBloss,focal_loss,AP-0,AP-1,time
0,0.489999,0.346516,0.647912,0.106673,0.239843,0.781901,0.513923,04:00
1,0.320945,0.513727,0.479651,0.100170,0.413557,0.647080,0.312222,03:55
2,0.270833,0.533050,0.505559,0.075589,0.457461,0.731166,0.279952,03:58
3,0.261638,0.357657,0.539344,0.078625,0.279033,0.766222,0.312467,04:02
4,0.227061,0.257097,0.728838,0.063724,0.193372,0.843238,0.614439,03:59
5,0.208701,0.315158,0.625346,0.066460,0.248698,0.802558,0.448133,04:00
6,0.195837,0.232755,0.699097,0.048763,0.183992,0.805275,0.592918,03:58
7,0.172495,0.242141,0.682297,0.044232,0.197908,0.814101,0.550493,04:01
8,0.155929,0.272521,0.660700,0.044176,0.228344,0.804521,0.516878,04:01
9,0.150222,0.226151,0.693598,0.043443,0.182707,0.816393,0.570803,04:00


this Learner object self-destroyed - it still exists, but no longer usable
